In [1]:
from __future__ import division
import datetime
import matplotlib.pyplot as plt
import cPickle as pickle
import numpy as np
import v2gsim 
import pandas 

# ### Require gurobi or CPLEX #####
# Create a project and initialize it with someitineraries
project = v2gsim.model.Project()
project = v2gsim.itinerary.from_excel(project, '../data/NHTS/Tennessee.xlsx')
project = v2gsim.itinerary.copy_append(project, nb_of_days_to_add=2)

# This function from the itinerary module return all the vehicles that
# start and end their day at the same location (e.g. home)
project.vehicles = v2gsim.itinerary.get_cycling_itineraries(project)


# Reduce the number of vehicles
project.vehicles = project.vehicles[0:100]


In [2]:
# Create some new charging infrastructures, append those new
# infrastructures to the project list of infrastructures
charging_stations = []
charging_stations.append(
    v2gsim.model.ChargingStation(name='L2', maximum_power=7200, minimum_power=0))
charging_stations.append(
    v2gsim.model.ChargingStation(name='L1_V1G', maximum_power=1400, minimum_power=0, post_simulation=True))
charging_stations.append(
    v2gsim.model.ChargingStation(name='L2_V2G', maximum_power=7200, minimum_power=-7200, post_simulation=True))
project.charging_stations.extend(charging_stations)

 #Create a data frame with the new infrastructures mix and
# apply this mix at all the locations
df = pandas.DataFrame(index=['L2', 'L1_V1G', 'L2_V2G'],
                      data={'charging_station': charging_stations,
                            'probability': [0.0, 0.4, 0.6]})
for location in project.locations:
    if location.category in ['Work', 'Home']:
        location.available_charging_station = df.copy()

# Initiate SOC and charging infrastructures
v2gsim.core.initialize_SOC(project, nb_iteration=2)

# Assign a basic result function to save power demand
for vehicle in project.vehicles:
    vehicle.result_function = v2gsim.post_simulation.netload_optimization.save_vehicle_state_for_optimization


# Launch the simulation
v2gsim.core.run(project, date_from=project.date + datetime.timedelta(days=1),
                date_to=project.date + datetime.timedelta(days=2),
                reset_charging_station=False)


core.initialize_SOC: 100%|####################################################|
core.run:  26%|################                                               |       mean  mean_rate       std  std_rate
0  0.950000   0.000000  0.000000  0.000000
1  0.914688   0.035312  0.126707 -0.126707
2  0.914691  -0.000003  0.126678  0.000030

core.run: 100%|###############################################################|


In [3]:
# Look at the results
total_power_demand = v2gsim.post_simulation.result.total_power_demand(project)

# Optimization
myopti = v2gsim.post_simulation.netload_optimization.CentralOptimization(project, 10,
                                                                         project.date + datetime.timedelta(days=1),
                                                                         project.date + datetime.timedelta(days=2),
                                                                         minimum_SOC=0.1, maximum_SOC=0.95)

In [4]:
finalResult = pandas.DataFrame()
filename = '../data/netload/2025.pickle'
# with open(filename, 'rb') as fp:
#     net_load = pickle.load(fp)
net_load=pandas.read_pickle(filename)
day = datetime.datetime(2025, 6, 17)
net_load = pandas.DataFrame(net_load[day: day + datetime.timedelta(days=1)]['netload'])
net_load.head()
net_load.shape

net_load.to_csv('../UCED/netload.csv')
pr=pandas.read_csv("../UCED/out_camb_R1_2018_price.csv")

In [5]:
myresult, u1result = myopti.solve(project, net_load * 1000000,1500000, price=pr,
                        peak_shaving='economic', SOC_margin=0.05)

There is 95 vehicle participating in the optimization (95.0%)
There is 5 unfeasible vehicle.

Solver script file: 'c:\users\meiye'~1\appdata\local\temp\tmpisveya.gurobi.script'
Solver log file: 'c:\users\meiye'~1\appdata\local\temp\tmpq1u24t.gurobi.log'
Solver solution file: 'c:\users\meiye'~1\appdata\local\temp\tmpppn66h.gurobi.txt'
Solver problem files: ("c:\\users\\meiye'~1\\appdata\\local\\temp\\tmpb_agkg.pyomo.lp",)
Academic license - for non-commercial use only
Read LP format model from file c:\users\meiye'~1\appdata\local\temp\tmpb_agkg.pyomo.lp
Reading time = 6.02 seconds
x82081: 41041 rows, 82081 columns, 68401 nonzeros
Optimize a model with 41041 rows, 82081 columns and 68401 nonzeros
Model has 41040 quadratic objective terms
Model has 27455 quadratic constraints
Variable types: 41041 continuous, 41040 integer (41040 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e+00, 

ApplicationError: Solver (gurobi) did not exit normally

In [6]:
print(pr)

Unnamed: 0  Hour    pr_e  pr_fre_u  pr_fre_d
0              0     0  18.665     17.08    16.446
1              1     1  18.665     17.08    16.446
2              2     2  18.665     17.08    16.446
3              3     3  18.665     17.08    16.446
4              4     4  18.665     17.08    16.446
5              5     5  18.665     17.08    16.446
6              6     6  18.665     17.08    16.446
7              7     7  18.665     17.08    16.236
8              8     8  18.665     17.08    16.236
9              9     9  18.665     17.08    16.236
10            10    10  18.665     17.08    16.236
11            11    11  18.665     17.08    16.236
12            12    12  18.665     17.08    16.236
13            13    13  18.665     17.08    16.446
14            14    14  18.665     17.08    16.236
15            15    15  18.665     17.08    16.236
16            16    16  18.665     17.08    16.236
17            17    17  18.665     17.08    16.236
18            18    18  18.665     17

In [ ]:
u1, ub, c1, cb, c2, cb2 ,i = u1result
u1pd = pandas.DataFrame(index=['power'], data=u1).transpose()
# u1pd.head(20)
# u1pd.shape
ubpd=pandas.DataFrame(index=['power binary'], data=ub).transpose()
# ubpd.shape
# ubpd.head(20)
u=pandas.concat([u1pd,ubpd],axis=1)

In [ ]:
u['powerresult']=u['power']*u['power binary']
u.head(20)

# powerresult=u['powerresult'].groupby(level=0).sum()

In [ ]:
discharge=u[u['power']>=0]
# discharge.head()
# discharge.shape
discharge = discharge.groupby(level=0).sum()
# discharge.rename(columns={'power':'power discharge'},inplace=True)
discharge.head()
# discharge.shape
# powerresult=pandas.DataFrame()
# powerresult=pandas.concat([powerresult,discharge],axis=1)
# powerresult.head()
# powerresult.shape


In [ ]:
charge=u[u['powerresult']<0]
charge.head()
# charge = charge['powerresult'].groupby(level=0).sum()

In [ ]:
charge.head()

In [ ]:
charge=u[u['power']<0]
charge.head()
# charge.shape

In [ ]:
pandas.DataFrame(data=np.ones(13680)).set_index?
# pandas.DataFrame(data=np.ones(13680)).head()

In [ ]:
charge = charge.groupby(level=0).sum()
charge.rename(columns={'power':'power charge'},inplace=True)
charge.head()
charge.shape

In [ ]:
powerresult=pandas.concat([powerresult,charge],axis=1)
powerresult.head()
# powerresult.shape

In [ ]:

df = pandas.DataFrame(index=['power discharged binary'], data=model.ub.get_values()).transpose().groupby(level=0).sum()
power = pandas.concat([power, df], axis=1)
       
df = pandas.DataFrame(index=['power regulation up'], data=model.c1.get_values()).transpose().groupby(level=0).sum()
power = pandas.concat([power, df], axis=1)
        
df = pandas.DataFrame(index=['power regulation up binary'], data=model.cb.get_values()).transpose().groupby(level=0).sum()
power = pandas.concat([power, df], axis=1)
        
df = pandas.DataFrame(index=['power regulation down'], data=model.c2.get_values()).transpose().groupby(level=0).sum()
power = pandas.concat([power, df], axis=1)
        
df = pandas.DataFrame(index=['power regulation down binary'], data=model.cb2.get_values()).transpose().groupby(level=0).sum()
power = pandas.concat([power, df], axis=1)

In [ ]:
powerresult =[]

dischar=[]
char=[]
regup=[]
regdown=[]
powersum=[]

for i in range(len(df)):           
    dischar.append(power.loc[[i],'power discharged']*power.loc[[i],'power discharged binary'])
    char.append(power.loc[[i],'power charged']*power.loc[[i],'power discharged binary'])
    regup.append(power.loc[[i],'power regulation up']*power.loc[[i],'power regulation up binary'])
    regdown.append(power.loc[[i],'power regulation down']*power.loc[[i],'power regulation down binary'])
    powersum.append(dischar[i]-char[i])
    powerresult.append((dischar[i],char[i],regup[i],regdown[i],powersum[i]))
        
powerresult_pd=pandas.DataFrame(powerresult,columns=('Discharge','Charge','Regup','Regdown','EnergySum'))


In [ ]:
powerresult


In [ ]:
temp2 = pandas.DataFrame(index=['vehicle_after'], data=powerresult_pd['EnergySum'])
temp2.head()

In [ ]:
powerresult.head()

In [ ]:

# dischar=np.array(powerresult.loc[:,'power discharged'])
# char=np.array(powerresult.loc[:,'power charged'])
 
powersum={}
power=[]

dischar=[]
for i in range(len(df)):           
    dischar[i]=powerresult.loc[[i],'power discharged']*powerresult.loc[[i],'power charged']
    char[i]=powerresult.loc[[i],'power discharged']+powerresult.loc[[i],'power charged']
    powersum[i]=dischar[i]+char[i]
    power.append((dischar[i],char[i],powersum[i]))



In [ ]:
 powerresult =[]

        dischar=[]
        char=[]
        regup=[]
        regdown=[]
        powersum=[]

        for i in range(len(df)):           
            dischar.append(power.loc[[i],'power discharged']*power.loc[[i],'power discharged binary'])
            char.append(power.loc[[i],'power charged']*power.loc[[i],'power discharged binary'])
            regup.append(power.loc[[i],'power regulation up']*power.loc[[i],'power regulation up binary'])
            regdown.append(power.loc[[i],'power regulation down']*power.loc[[i],'power regulation down binary'])
            powersum.append(dischar[i]-char[i])
            powerresult.append((dischar[i],char[i],regup[i],regdown[i],powersum[i]))
        
        powerresult_pd=pandas.DataFrame(powerresult,columns=('Discharge','Charge','Regup','Regdown','EnergySum'))
        
        
        for vehicle in project.vehicles:
            if vehicle.result is not None:
                if first:
                    temp['vehicle_before'] = vehicle.result['power_demand']
                    first = False
                else:
                    temp['vehicle_before'] += vehicle.result['power_demand']

        temp2 = pandas.DataFrame(index=['vehicle_after'], data=powerresult_pd['EnergySum'])
        i = pandas.date_range(start=self.date_from, end=self.date_to,
                              freq=str(self.optimization_timestep) + 'T', closed='left')

        temp2 = temp2.set_index(i)
        temp2 = temp2.resample(str(project.timestep) + 'S')
        temp2 = temp2.fillna(method='ffill').fillna(method='bfill')

        temp3 = pandas.DataFrame(index=['vehicle_after_demand'], data=powerresult_pd['Charge'])
        i = pandas.date_range(start=self.date_from, end=self.date_to,
                              freq=str(self.optimization_timestep) + 'T', closed='left')
        temp3 = temp3.set_index(i)
        temp3 = temp3.resample(str(project.timestep) + 'S')
        temp3 = temp3.fillna(method='ffill').fillna(method='bfill')

        temp4 = pandas.DataFrame(index=['vehicle_after_generation'], data=powerresult_pd['Discharge'])
        i = pandas.date_range(start=self.date_from, end=self.date_to,
                              freq=str(self.optimization_timestep) + 'T', closed='left')
        temp4 = temp4.set_index(i)
        temp4 = temp4.resample(str(project.timestep) + 'S')
        temp4 = temp4.fillna(method='ffill').fillna(method='bfill')

        final_result = pandas.DataFrame()
        final_result = pandas.concat([temp['vehicle_before'], temp2['vehicle_after']],temp3['vehicle_after_demand'],temp4['vehicle_after_generatio'],axis=1)
        final_result = final_result.fillna(method='ffill').fillna(method='bfill')


In [ ]:
power_pd=pandas.DataFrame(power,columns=('Discharge','Charge','EnergySum'))
power_pd['EnergySum']

In [ ]:
#Get the result in the right format
temp_vehicle = pandas.DataFrame(
    (total_power_demand['total'] - myresult['vehicle_before'] + myresult['vehicle_after']) *
    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle = temp_vehicle.rename(columns={0: 'vehicle'})
temp_vehicle['index'] = range(0, len(temp_vehicle))
temp_vehicle = temp_vehicle.set_index(['index'], drop=True)

temp_netload = net_load.copy()
temp_netload = temp_netload.resample('60S')
temp_netload = temp_netload.fillna(method='ffill').fillna(method='bfill')
temp_netload = temp_netload.head(len(temp_vehicle))
tempIndex = temp_netload.index
temp_netload['index'] = range(0, len(temp_vehicle))
temp_netload = temp_netload.set_index(['index'], drop=True)

temp_result = pandas.DataFrame(temp_netload['netload'] + temp_vehicle['vehicle'])
temp_result = temp_result.rename(columns={0: 'netload'})
temp_result = temp_result.set_index(tempIndex)
temp_netload = temp_netload.set_index(tempIndex)
temp_vehicle = temp_vehicle.set_index(tempIndex)

In [ ]:

plt.plot(temp_netload['netload'], label='netload')
plt.plot(temp_result['netload'], label='netload + vehicles')
plt.ylabel('Power (MW)')
plt.legend()
plt.show()


In [ ]:
# temp_netload
temp_vehicle

In [ ]:
myopti.plot_result()